In [1]:
# Load modules:
import numpy as np
import pprint
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
import xarray as xr
import cmocean as cm
from scipy import interpolate

import matplotlib.path as mpath
import cartopy.crs as ccrs
import cartopy.feature as cft

import netCDF4
import os, sys, glob
%matplotlib inline
os.chdir('/home/561/rmh561/Radiation_CMIP_Practical')

import xesmf

## Description:

This script processes CMIP6 output from NCI's repository (https://opus.nci.org.au/display/CMIP/Overview+on+CMIP) into standardsized data formats for use in the GEOS2115/2915 Practicals.

It also processes some observational data.

Model data is processed for the following variables:
- Surface Air Temperature (tas)
- SW up, SW down and LW up at TOA.
- LW down and LW up at surface.
- Precipitation
- Sea level data

Data is processed from the following experiments:
- Historical
- SSP245 (middle-of-the-road)
- SSP585 (fossil fueled development)

Data is processed from the following models:
- The Australian Community Climate and Earth System Simulator Climate Model Version 2.0 (ACCESS-CM2), run by CSIRO in Australia, CS = 4.7
- The Max Plank Institute (Germany)'s Earth System Model (MPI-ESM), CS = 3.0
- The Canadian model CanESM5, CS = 5.6.

Some code is present to also process the following models:
- The Geophysical Fluid Dynamics Laboratory (NOAA-GFDL) Climate Model Version 4.0 (GFDL-CM4), run by the National Oceanic and Atmospheric Administration (NOAA) in the United States, CS = 3.9
(GFDL is missing zostoga)
- The Community Earth System Model Version 2.0 (CESM2), run by the National Center for Atmospheric Research (NCAR) in the United States, CS = 5.2
(CESM2 is missing lots of stuff...)

Data is processed into annual averages to reduce data costs.

Meta data is cleaned up so that the students don't have to worry too much about things like units, time etc. etc.

In [ ]:
# CMIP6 data paths:
models = {'ACCESS-CM2': {'base':'/g/data/fs38/publications/CMIP6', 'org':'CSIRO-ARCCSS','model':'ACCESS-CM2','ens':'r2i1p1f1','v1':'gn','v2':'latest','name':'ACCESS-CM2'},
          'ACCESS-CM2-Ensemble-2': {'base':'/g/data/fs38/publications/CMIP6', 'org':'CSIRO-ARCCSS','model':'ACCESS-CM2','ens':'r3i1p1f1','v1':'gn','v2':'latest','name':'ACCESS-CM2-E2'},
          'MPI-ESM': {'base':'/g/data/oi10/replicas/CMIP6', 'org':'MPI-M','model':'MPI-ESM1-2-LR','ens':'r1i1p1f1','v1':'gn','v2':'v20190710','name':'MPI-ESM'},
          'CanESM5': {'base':'/g/data/oi10/replicas/CMIP6', 'org':'CCCma','model':'CanESM5','ens':'r1i1p1f1','v1':'gn','v2':'v20190429','name':'CanESM5'},
#          'GFDL-CM4': {'base':'/g/data/oi10/replicas/CMIP6', 'org':'NOAA-GFDL','model':'GFDL-CM4','ens':'r1i1p1f1','v1':'gr1','v2':'v20180701'},
#          'CESM2': {'base':'/g/data/oi10/replicas/CMIP6', 'org':'NCAR','model':'CESM2','ens':'r1i1p1f1'}, # not much data
         }

# Variables:
# SAT: tas
# SW down surface: rsds
# SW up surface: rsus
atm_vars = {'tas':'SAT',
            'rsdt':'SWdownTOA',
            'rsut':'SWupTOA',
            'rlut':'LWupTOA',
            'rlds':'LWdownSFC',
            'rlus':'LWupSFC',
            'pr':'PRCP'
           }

time_limits = '1800-01-01','2100-12-31'

ocn_vars = {'zostoga':'GSL'}

all_vars = atm_vars.copy()
all_vars.update(ocn_vars)

scenarios = {'hist':{'name':'historical','MIP':'CMIP'},
             'ssp245':{'name':'ssp245','MIP':'ScenarioMIP'},
             'ssp585':{'name':'ssp585','MIP':'ScenarioMIP'}}            

In [ ]:
def calc_area(ds):
    """
    Function to calculate area from an xarray CMIP6 object
    """
    dlon = ds.longitude.diff('longitude').isel(longitude=0).values
    dlat = np.abs(ds.latitude.diff('latitude').isel(latitude=0).values)
    area = (1.113e5)**2*dlon*dlat*np.cos(ds['latitude']/180.*np.pi)
    area,lon = xr.broadcast(area,ds.longitude)
    return(area)

In [ ]:
# Check file names and date ranges:
model_keys = models.keys()
scenario_keys = scenarios.keys()

for model in model_keys:
    for scenario in scenario_keys:
        
        # File paths:
        mpaths = models[model]
        scn = scenarios[scenario]
        mname = mpaths['model']
        base = mpaths['base'] + '/' + scn['MIP'] + '/' + mpaths['org'] + '/' + mpaths['model'] + '/' + scn['name'] + '/' + mpaths['ens'] + '/'
        
        print(model + ' ' + scenario + '(base ' + base + ' :')
        for var in atm_vars.keys():
            name = atm_vars[var]
            files = glob.glob(base + 'Amon/' + var + '/' + mpaths['v1'] + '/' + mpaths['v2'] + '/*.nc')
            for file in files:
                print('                ' + var + ': ' + file)
        for var in ocn_vars.keys():
            name = ocn_vars[var]
            files = glob.glob(base + 'Omon/' + var + '/' + mpaths['v1'] + '/' + mpaths['v2'] + '/*.nc')
            for file in files:
                print('                ' + var + ': ' + file)

In [ ]:
# Process atmospheric grid data:
model_keys = ['ACCESS-CM2','ACCESS-CM2-Ensemble-2','MPI-ESM','CanESM5']
scenario_keys = ['hist','ssp245','ssp585']

for model in model_keys:
    for scenario in scenario_keys:
        
        # File paths:
        mpaths = models[model]
        scn = scenarios[scenario]
        mname = mpaths['model']
        base = mpaths['base'] + '/' + scn['MIP'] + '/' + mpaths['org'] + '/' + mpaths['model'] + '/' + scn['name'] + '/' + mpaths['ens'] + '/'
        file_out = 'data/' + mpaths['name'] + '_' + scenario + '_data.nc'
        
        vars_list = []
        for var in atm_vars.keys():
            name = atm_vars[var]
            files = glob.glob(base + 'Amon/' + var + '/' + mpaths['v1'] + '/' + mpaths['v2'] + '/*.nc')
            print('Doing ' + model + ' ' + scenario + ' ' + name)
            try:
                var = xr.open_mfdataset(files).rename({'lat':'latitude','lon':'longitude'})
            except:
                var = xr.open_mfdataset(files,use_cftime=True).rename({'lat':'latitude','lon':'longitude'})
            var = var.sel(time=slice(time_limits[0],time_limits[1]))
            var = var.resample(time='A').mean().load()
            vars_list.append(var)
        
        # Global sea level:
        files = glob.glob(base + 'Omon/zostoga/' + mpaths['v1'] + '/' + mpaths['v2'] + '/*.nc')
        try:
            zostoga = xr.open_mfdataset(files)
        except:
            zostoga = xr.open_mfdataset(files,use_cftime=True)
        print('Doing ' + model + ' ' + scenario + ' sea level')
        zostoga = zostoga.sel(time=slice(time_limits[0],time_limits[1]))
        zostoga = zostoga.resample(time='A').mean().load() 
        vars_list.append(zostoga)
        
        print('Saving to file ' + model + ' ' + scenario)
        # Combine atmospheric variables into a single dataset:
        ds = xr.merge(vars_list)
        
        # Calculate area:
        ds['area'] = calc_area(ds)
        
        # Drop and rename variables:
        ds = ds.drop_vars(['lat_bnds','lon_bnds','height'])
        ds = ds.rename(all_vars)
        
        # Save to netcdf file:
        ds.to_netcdf(file_out)

In [ ]:
def global_average(variable,area):
    """
    Calculate the global average of a variable using an equal-area weighting
    """
    total_area = area.where(variable.notnull()).sum('longitude').sum('latitude')
    average = (variable*area).sum('longitude').sum('latitude')/total_area
    return(average)

In [ ]:
# Interpolate sea level to atmospheric grid and then save back into same file:
model_keys = ['MPI-ESM','CanESM5']
scenario_keys = ['hist','ssp245','ssp585']

for model in model_keys:
    for scenario in scenario_keys:
        
        # File paths:
        mpaths = models[model]
        scn = scenarios[scenario]
        mname = mpaths['model']
        base = mpaths['base'] + '/' + scn['MIP'] + '/' + mpaths['org'] + '/' + mpaths['model'] + '/' + scn['name'] + '/' + mpaths['ens'] + '/'
        file_out = 'data/' + mpaths['name'] + '_' + scenario + '_data.nc'
        
        # Load pre-processed file:
        ds = xr.open_dataset(file_out)
        
        # Load regional sea level on ocean grid and resample to annual values:
        print('Loading ' + model + ' ' + scenario + ' sea level')
        files = glob.glob(base + 'Omon/zos/' + mpaths['v1'] + '/' + mpaths['v2'] + '/*.nc')
        try:
            zos = xr.open_mfdataset(files)
        except:
            zos = xr.open_mfdataset(files,use_cftime=True)
        zos = zos.resample(time='A').mean().load()
        
        # Interpolate to atmospheric grid:
        print('Interpolating ' + model + ' ' + scenario + ' sea level')
        zos = zos.rename({'i':'x','j':'y'}).drop({'x','y'}).zos
        regridder = xesmf.Regridder(zos, ds, 'bilinear', periodic=True,
                                               filename="data/tmp.nc")
        ds['SL'] = regridder(zos.chunk({'x': None, 'y':None}))
        
        # subtract and add global mean:
        GSL = global_average(ds.SL,ds.area)
        ds['SL'] = ds.SL - GSL + ds.GSL
        
        print('Saving to file ' + model + ' ' + scenario)     
        
        # Save back into same netcdf file:
        os.rename(file_out,file_out[:-2]+ 'backup.nc')
        ds.to_netcdf(file_out)

# Observational data:

Here we pre-process some observational data.

Data includes:
- Surface air temperature covering 1900-2010 from the 20th century European Reanalysis (ERA-20C).
- Time-mean SW and LW ls /g/datdata from JRA-55?
- Time-mean precipitation maps from JRA-55? Any point in looking at observational precip (e.g. TRMM)?
- CMEMS sea level?

In [ ]:
# ERA 20th century reanalysis:
ERA20C = xr.open_dataset('/g/data/ua8/ERA20C/skt_era20c_mon_1900_2010.nc').rename({'skt':'tas'}).tas
ERA20C = ERA20C.resample(time='A').mean()
ERA20C["area"] = calc_area(ERA20C)

ERA20C = ERA20C.rename('SAT')

# Write out to file:
ERA20C.to_netcdf('data/ERA20C_hist.nc')

## Meta-data/units post-corrections

In [7]:
!ls data/

ACCESS-CM2-E2_hist_data.nc    ACCESS-CM2_ssp585_data.nc  ERA20C_hist_data.nc
ACCESS-CM2-E2_ssp245_data.nc  backups			 ERA20C_hist.nc
ACCESS-CM2-E2_ssp585_data.nc  CanESM5_hist_data.nc	 MPI-ESM_hist_data.nc
ACCESS-CM2_hist_data.nc       CanESM5_ssp245_data.nc	 MPI-ESM_ssp245_data.nc
ACCESS-CM2_ssp245_data.nc     CanESM5_ssp585_data.nc	 MPI-ESM_ssp585_data.nc


In [58]:
models = ['ACCESS-CM2']#,'CanESM5','MPI-ESM']
scenars = ['hist','ssp245','ssp585']

for model in models:
    for scenar in scenars:

        file = 'data/' + model + '_' + scenar + '_data.nc'
        file_out = file[:-3] + '_cor.nc'

        ds = xr.open_dataset(file)

        # Edit time to be just an integer year and rename it:
        ds["time"] = ds.time.dt.year
        ds = ds.rename({'time':'year'})

        # Change some units:
        ds.SAT.values = ds.SAT.values - 273.15
        ds.SAT.attrs['units'] = 'degrees celsius'

        ds.PRCP.values = ds.PRCP.values/1000.*(86400.*1000.)
        ds.PRCP.attrs['units'] = 'mm/day'

        # Change some names:
        ds.SL.attrs['standard_name'] = 'Sea Level'
        ds.SL.attrs['long_name'] = 'Sea Level'
        ds.SL.attrs['units'] = 'm'
        ds.SL.attrs['comment'] = 'Calculated from relative sealevel (zos) + global mean thermosteric sea level (zostoga)'

        ds.GSL.attrs['long_name'] = 'Global Average Sea Level'

        ds.area.attrs['long_name'] = 'Grid Cell Area'
        ds.area.attrs['units'] = 'm^2'

        ds.to_netcdf(file_out)

In [4]:
# Do the same for the observational data:
file = 'data/ERA20C_hist.nc'
file_out = file[:-3] + '_cor.nc'
ds = xr.open_dataset(file)

# Edit time to be just an integer year and rename it:
ds["time"] = ds.time.dt.year
ds = ds.rename({'time':'year'})

# Change some units:
ds.SAT.values = ds.SAT.values - 273.15
ds.SAT.attrs['units'] = 'degrees celsius'
ds.SAT.attrs['long_name'] = 'Near-Surface Air Temperature'
ds.SAT.attrs['standard_name'] = 'air_temperature'

ds.to_netcdf(file_out)

## Example/old plotting from the above data

In [ ]:
# Plot global mean surface temperature:
ERA20C_GMST = (ERA20C_tas.tas*ERA20C_tas.area).sum('longitude').sum('latitude')/ERA20C_tas.area.sum('longitude').sum('latitude')
(ERA20C_GMST-273.15).plot(label='ERA 20th Century Reanalysis (observations)')
CM2_hist_GMST = (CM2_hist_tas.tas*CM2_hist_tas.area).sum('longitude').sum('latitude')/CM2_hist_tas.area.sum('longitude').sum('latitude')
(CM2_hist_GMST-273.15).plot(label='ACCESS-CM2 historical')
CM2_hist_ens2_GMST = (CM2_hist_ens2_tas.tas*CM2_hist_ens2_tas.area).sum('longitude').sum('latitude')/CM2_hist_ens2_tas.area.sum('longitude').sum('latitude')
(CM2_hist_ens2_GMST-273.15).plot(label='ACCESS-CM2 historical ensemble 2')
GFDLCM4_hist_GMST = (GFDLCM4_hist_tas.tas*GFDLCM4_hist_tas.area).sum('longitude').sum('latitude')/GFDLCM4_hist_tas.area.sum('longitude').sum('latitude')
(GFDLCM4_hist_GMST-273.15).plot(label='GFDL-CM4 historical')
plt.xlabel('Year')
plt.ylabel('Global Mean Surface Temperature ($^\circ$C)')
plt.title('')
plt.legend()

In [ ]:
# Plot Spatial structure:
plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
(ERA20C_tas.tas-273.15).sel(time=slice('2000-01-01','2010-12-31')).mean('time').plot(vmin=0.,vmax=30.,cmap='RdBu_r')

plt.subplot(2,2,2)
(ERA20C_tas.tas-273.15).sel(time=slice('1900-01-01','1910-12-31')).mean('time').plot(vmin=0.,vmax=30.,cmap='RdBu_r')

plt.subplot(2,2,3)
((ERA20C_tas.tas-273.15).sel(time=slice('2000-01-01','2010-12-31')).mean('time')-(ERA20C_tas.tas-273.15).sel(time=slice('1900-01-01','1910-12-31')).mean('time')).plot(vmin=-3.,vmax=3.,cmap='RdBu_r')

In [ ]:
# Plot near Sydney:
(ERA20C_tas.tas-273.15).sel(longitude=151.,method='nearest').sel(latitude=-33.5,method='nearest').plot(label='Sydney')
(ERA20C_tas.tas-273.15).sel(longitude=116.,method='nearest').sel(latitude=40.,method='nearest').plot(label='Beijing')
(ERA20C_tas.tas-273.15).sel(longitude=(360.-74.),method='nearest').sel(latitude=40.5,method='nearest').plot(label='New York')
plt.legend()

In [ ]:
# Plot zonal mean:
ERA20C_tas_ZM = (ERA20C_tas.tas-273.15).mean('longitude')
(ERA20C_tas_ZM-ERA20C_tas_ZM.sel(time=slice('1900-01-01','1920-12-31')).mean('time')).plot(vmin=-2.,vmax=2.,cmap='RdBu_r')

# CMIP models:

In [ ]:
# Data preprocessing:
# ACCESS-CM2 historical ensemble 1:
#tas = xr.open_dataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/tas/gn/latest/tas_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.nc').rename({'lat':'latitude','lon':'longitude'})
# ACCESS-CM2 SSP245:
# tas = xr.open_dataset('/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp245/r1i1p1f1/Amon/tas/gn/latest/tas_Amon_ACCESS-CM2_ssp245_r1i1p1f1_gn_201501-210012.nc').rename({'lat':'latitude','lon':'longitude'})
# ACCESS-CM2 SSP585:
tas = xr.open_dataset('/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp585/r1i1p1f1/Amon/tas/gn/latest/tas_Amon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.nc').rename({'lat':'latitude','lon':'longitude'})
# ACCESS-CM2 historical ensemble 2:
# tas = xr.open_dataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r2i1p1f1/Amon/tas/gn/latest/tas_Amon_ACCESS-CM2_historical_r2i1p1f1_gn_185001-201412.nc').rename({'lat':'latitude','lon':'longitude'})
# GFDL CM4 historical:
# tas = xr.open_mfdataset('/g/data/oi10/replicas/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/Amon/tas/gr1/v20180701/tas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_1*.nc').rename({'lat':'latitude','lon':'longitude'})
# GFDL CM4 SSP245:
# tas = xr.open_dataset('/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp245/r1i1p1f1/Amon/tas/gr1/v20180701/tas_Amon_GFDL-CM4_ssp245_r1i1p1f1_gr1_201501-210012.nc').rename({'lat':'latitude','lon':'longitude'})

tas = tas.resample(time='A').mean()

# Add area:
dlon = tas.longitude.diff('longitude').isel(longitude=0).values
dlat = np.abs(tas.latitude.diff('latitude').isel(latitude=0).values)
area = (1.113e5)**2*dlon*dlat*np.cos(tas['latitude']/180.*np.pi)
area,lon = xr.broadcast(area,tas.longitude)
tas['area'] = area

# Write out to file:
tas.to_netcdf('CM2_ssp585_tas.nc')

In [ ]:
# load pre-processed data:
CM2_hist_tas = xr.open_dataset('CM2_hist_tas.nc')
CM2_ssp245_tas = xr.open_dataset('CM2_ssp245_tas.nc')
CM2_ssp585_tas = xr.open_dataset('CM2_ssp585_tas.nc')
CM2_hist_ens2_tas = xr.open_dataset('CM2_hist_ens2_tas.nc')
GFDLCM4_hist_tas = xr.open_dataset('GFDLCM4_hist_tas.nc')
GFDLCM4_ssp245_tas = xr.open_dataset('GFDLCM4_ssp245_tas.nc')
ERA20C_tas = xr.open_dataset('ERA20C_tas.nc')

In [ ]:
# Plot global mean surface temperature:
plt.figure(figsize=(20,10))
ERA20C_GMST = (ERA20C_tas.tas*ERA20C_tas.area).sum('longitude').sum('latitude')/ERA20C_tas.area.sum('longitude').sum('latitude')
(ERA20C_GMST-ERA20C_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='ERA 20th Century Reanalysis (observations)')
CM2_hist_GMST = (CM2_hist_tas.tas*CM2_hist_tas.area).sum('longitude').sum('latitude')/CM2_hist_tas.area.sum('longitude').sum('latitude')
(CM2_hist_GMST-CM2_hist_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='ACCESS-CM2 historical')
CM2_hist_ens2_GMST = (CM2_hist_ens2_tas.tas*CM2_hist_ens2_tas.area).sum('longitude').sum('latitude')/CM2_hist_ens2_tas.area.sum('longitude').sum('latitude')
(CM2_hist_ens2_GMST-CM2_hist_ens2_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='ACCESS-CM2 historical ensemble 2')
GFDLCM4_hist_GMST = (GFDLCM4_hist_tas.tas*GFDLCM4_hist_tas.area).sum('longitude').sum('latitude')/GFDLCM4_hist_tas.area.sum('longitude').sum('latitude')
(GFDLCM4_hist_GMST-GFDLCM4_hist_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='GFDL-CM4 historical')
CM2_ssp245_GMST = (CM2_ssp245_tas.tas*CM2_hist_tas.area).sum('longitude').sum('latitude')/CM2_ssp245_tas.area.sum('longitude').sum('latitude')
(CM2_ssp245_GMST-CM2_hist_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='ACCESS-CM2 SSP245')
GFDLCM4_ssp245_GMST = (GFDLCM4_ssp245_tas.tas*GFDLCM4_hist_tas.area).sum('longitude').sum('latitude')/GFDLCM4_ssp245_tas.area.sum('longitude').sum('latitude')
(GFDLCM4_ssp245_GMST-GFDLCM4_hist_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='GFDL-CM4 SSP245')
CM2_ssp585_taST = (CM2_ssp585_tas.tas*CM2_hist_tas.area).sum('longitude').sum('latitude')/CM2_ssp585_tas.area.sum('longitude').sum('latitude')
(CM2_ssp585_taST-CM2_hist_GMST.sel(time=slice('1850-01-01','1900-01-01')).mean('time')).plot(label='ACCESS-CM2 SSP585')
plt.xlabel('Year')
plt.ylabel('Global Mean Surface Temperature ($^\circ$C)')
plt.title('')
plt.legend()

In [ ]:
# Plot Spatial structure of epoch differences:
plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
((CM2_hist_tas.tas-273.15).sel(time=slice('2000-01-01','2010-12-31')).mean('time')-(CM2_hist_tas.tas-273.15).sel(time=slice('1900-01-01','1910-12-31')).mean('time')).plot(vmin=-3.,vmax=3.,cmap='RdBu_r')
plt.title('ACCESS-CM2')

plt.subplot(2,2,2)
((ERA20C_tas.tas-273.15).sel(time=slice('2000-01-01','2010-12-31')).mean('time')-(ERA20C_tas.tas-273.15).sel(time=slice('1900-01-01','1910-12-31')).mean('time')).plot(vmin=-3.,vmax=3.,cmap='RdBu_r')
plt.title('ERA20C')

plt.subplot(2,2,3)
((GFDLCM4_hist_tas.tas-273.15).sel(time=slice('2000-01-01','2010-12-31')).mean('time')-(GFDLCM4_hist_tas.tas-273.15).sel(time=slice('1900-01-01','1910-12-31')).mean('time')).plot(vmin=-3.,vmax=3.,cmap='RdBu_r')
plt.title('GFDL-CM4')

In [ ]:
# Plot Spatial structure of epoch differences:
plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
((CM2_ssp245_tas.tas-273.15).sel(time=slice('2090-01-01','2100-12-31')).mean('time')-(CM2_ssp245_tas.tas-273.15).sel(time=slice('2020-01-01','2030-12-31')).mean('time')).plot(vmin=-5.,vmax=5.,cmap='RdBu_r')
plt.title('ACCESS-CM2 SSP245')

plt.subplot(2,2,2)
((CM2_ssp585_tas.tas-273.15).sel(time=slice('2090-01-01','2100-12-31')).mean('time')-(CM2_ssp585_tas.tas-273.15).sel(time=slice('2020-01-01','2030-12-31')).mean('time')).plot(vmin=-5.,vmax=5.,cmap='RdBu_r')
plt.title('ACCESS-CM2 SSP585')

plt.subplot(2,2,3)
((GFDLCM4_ssp245_tas.tas-273.15).sel(time=slice('2090-01-01','2100-12-31')).mean('time')-(GFDLCM4_ssp245_tas.tas-273.15).sel(time=slice('2020-01-01','2030-12-31')).mean('time')).plot(vmin=-5.,vmax=5.,cmap='RdBu_r')
plt.title('GFDL-CM4 SSP245')

# HadISST

In [ ]:
# HadISST:
HadISST = xr.open_dataset('/g/data/ua8/HadISST/v1-1/HadISST_sst.nc')
dlon = HadISST.longitude.diff('longitude').isel(longitude=0).values
dlat = np.abs(HadISST.latitude.diff('latitude').isel(latitude=0).values)
area = (1.113e5)**2*dlon*dlat*np.cos(HadISST['latitude']/180.*np.pi)
area,lon = xr.broadcast(area,HadISST.longitude)
HadISST['area'] = area

# Adjust for transition at -180:
#HadISST = HadISST.roll(longitude=180)
#HadISST['longitude'] = np.where(HadISST.longitude>0,HadISST.longitude,HadISST.longitude+360.)

In [ ]:
(HadISST.sst.sel(time=slice('2000-01-01','2010-12-31')).mean('time')-HadISST.sst.sel(time=slice('1900-01-01','1910-12-31')).mean('time')).plot(vmin=-2.,vmax=2.,cmap='RdBu_r')